# Spark Structured Streaming - Demo
## Fire alarm

In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
import io
from pyspark.sql.functions import *
import time
import json
import struct
import requests 

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1,org.apache.spark:spark-streaming-kafka-0-10_2.11:2.4.5,org.apache.kafka:kafka-clients:2.6.0 pyspark-shell'
                                    
spark = (SparkSession.builder 
    .master("local[*]")
    .appName("test")
    .getOrCreate()
        )

spark

set up the environment variables

In [2]:
smoke_topic = 'SmokeSensorEvent'
temperature_topic = 'TemperatureSensorEvent'
servers = "kafka:9092"

## Understanding spark-kafka integration
Let's treat first kafka as a bulk source using `SparkSession.read()` instead of `SparkSession.readStream()` (see below)

In [3]:
smoke_df = (spark
  .read
  .format("kafka")
  .option("kafka.bootstrap.servers", servers)
  .option("subscribe", smoke_topic)
  .option("startingOffsets", "earliest")
  .option("endingOffsets", "latest")
  .load())

In [4]:
smoke_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
smoke_df.show(5)

+-------+--------------------+----------------+---------+------+--------------------+-------------+
|    key|               value|           topic|partition|offset|           timestamp|timestampType|
+-------+--------------------+----------------+---------+------+--------------------+-------------+
|[53 31]|[7B 22 73 65 6E 7...|SmokeSensorEvent|        0|     0|2024-10-14 08:13:...|            0|
+-------+--------------------+----------------+---------+------+--------------------+-------------+



In [8]:
stringified_smoke_df = smoke_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
stringified_smoke_df.show(5,False)

+---+--------------------------------------------------+
|key|value                                             |
+---+--------------------------------------------------+
|S1 |{"sensor": "S1", "smoke": false, "ts": 1728893631}|
+---+--------------------------------------------------+



In [9]:
stringified_smoke_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



In [10]:
from pyspark.sql.types import *

smoke_schema = StructType([
    StructField("sensor", StringType(), True),
    StructField("smoke", BooleanType(), True),
    StructField("ts", TimestampType(), True)])

In [11]:
smoke_df = stringified_smoke_df.select(col("key").cast("string"),from_json(col("value"), smoke_schema).alias("value"))

In [12]:
smoke_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: struct (nullable = true)
 |    |-- sensor: string (nullable = true)
 |    |-- smoke: boolean (nullable = true)
 |    |-- ts: timestamp (nullable = true)



In [18]:
smoke_df.select("value.*").show(5)

+------+-----+-------------------+
|sensor|smoke|                 ts|
+------+-----+-------------------+
|    S1|false|2024-10-14 08:13:51|
|    S1|false|2024-10-14 08:14:02|
|    S1|false|2024-10-14 08:14:12|
|    S1|false|2024-10-14 08:14:22|
|    S1|false|2024-10-14 08:14:32|
+------+-----+-------------------+
only showing top 5 rows



## Let's explore Spark Structured Streaming by example
Please refer to [EPL fire allarm](https://github.com/emanueledellavalle/streaming-data-analytics/tree/main/codes/epl_firealarm) for the EPL version of the following queries.

### Let's create the streaming Data Frames using the data in the kafka smoke topic

This time using the correct `SparkSession.readStream()`

In [14]:
raw_streaming_smoke_df = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", servers)
  .option("startingOffsets", "earliest")
  .option("subscribe", smoke_topic)
  .load())

In [15]:
raw_streaming_smoke_df.isStreaming

True

In [16]:
raw_streaming_smoke_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [19]:
smoke_sdf=(raw_streaming_smoke_df
                      .select(from_json(col("value").cast("string"), smoke_schema).alias("value"))
                      .select("value.*"))

In [20]:
smoke_sdf.printSchema()

root
 |-- sensor: string (nullable = true)
 |-- smoke: boolean (nullable = true)
 |-- ts: timestamp (nullable = true)



**NOTE**: it is not a DataFrame, you cannot directly execute an action on it. 

**The following cell *intetionally* gives an error**.

In [ ]:
smoke_sdf.count()

Queries with streaming sources must be registred and started with `writeStream.start()`

### Let's register and start a simple query

In [23]:
basic_query = (smoke_sdf
    .writeStream
    .format("memory") # this is for debug purpose only! DO NOT USE IN PRODUCTION
    .queryName("sinkTable")
    .start())

In [24]:
basic_query.lastProgress

run the following cell to see the most recent content of the sinkTable

In [25]:
spark.sql("SELECT * FROM sinkTable ORDER BY TS DESC").show(5)

+------+-----+-------------------+
|sensor|smoke|                 ts|
+------+-----+-------------------+
|    S1|false|2024-10-14 08:57:36|
|    S1|false|2024-10-14 08:57:26|
|    S1|false|2024-10-14 08:57:16|
|    S1|false|2024-10-14 08:57:06|
|    S1|false|2024-10-14 08:56:56|
+------+-----+-------------------+
only showing top 5 rows



do not forget to stop queries that you are not using

In [26]:
basic_query.stop()

### Let's create the streaming Data Frames for the kafka temperature topic

In [21]:
temperarture_schema = StructType([
    StructField("sensor", StringType(), True),
    StructField("temperature", DoubleType(), True),
    StructField("ts", TimestampType(), True)])

raw_streaming_temperature_df = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", servers)
  .option("startingOffsets", "earliest")
  .option("subscribe", temperature_topic)
  .load())

temperature_sdf = (raw_streaming_temperature_df
                      .select(from_json(col("value").cast("string"), temperarture_schema).alias("value"))
                      .select("value.*"))

In [22]:
temperature_sdf.printSchema()

root
 |-- sensor: string (nullable = true)
 |-- temperature: double (nullable = true)
 |-- ts: timestamp (nullable = true)



## Q0 - Filter

The temperature events whose temperature is greater than 50 °C 

### the SQL style

In [41]:
# create a logic table on top of the streaming data frame
temperature_sdf.createTempView("TemperatureSensorEvent")

# write your query in SQL, register it and start it
q0 = (spark.sql("select * from TemperatureSensorEvent where temperature > 50")
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

let's ask for the execution plan, we will compare it with cells down with the one of the query in Data Frame style

In [42]:
q0.explain()

== Physical Plan ==
WriteToDataSourceV2 org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@5df0a71c
+- Project [from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#333 as string), Some(Etc/UTC)).sensor AS sensor#348, from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#333 as string), Some(Etc/UTC)).temperature AS temperature#349, from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#333 as string), Some(Etc/UTC)).ts AS ts#350]
   +- Filter (from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#333 as string), Some(Etc/UTC)).temperature > 50.0)
      +- *(1) Project [key#332, value#333, topic#334, partition#335, offset#336L, timestamp#337, timesta

In [44]:
# look up the most recent results
spark.sql("SELECT * FROM sinkTable ORDER BY TS DESC").show(5)

+------+------------------+-------------------+
|sensor|       temperature|                 ts|
+------+------------------+-------------------+
|    S1| 55.70282805507498|2024-10-14 08:59:39|
|    S1| 57.08978801827377|2024-10-14 08:59:29|
|    S1| 53.25565016613896|2024-10-14 08:59:19|
|    S1| 53.30470577676184|2024-10-14 08:59:09|
|    S1|54.221264567328696|2024-10-14 08:58:59|
+------+------------------+-------------------+



if you are following carefully the instruction it should be empty because we are sending temperature aropund 20 °C.

Go back to the `temperature_sensor_simulator` notebook, stop the cell that is sending temperature around 20°C and run the one that sends temperature around 55°C

In [45]:
# look up the most recent results
spark.sql("SELECT * FROM sinkTable ORDER BY TS DESC").show(5)

+------+------------------+-------------------+
|sensor|       temperature|                 ts|
+------+------------------+-------------------+
|    S1| 54.61087713267025|2024-10-14 08:59:59|
|    S1|54.479731860672885|2024-10-14 08:59:49|
|    S1| 55.70282805507498|2024-10-14 08:59:39|
|    S1| 57.08978801827377|2024-10-14 08:59:29|
|    S1| 53.25565016613896|2024-10-14 08:59:19|
+------+------------------+-------------------+
only showing top 5 rows



Now you should see results.

In [46]:
# clean up
q0.stop()
spark.catalog.dropTempView("TemperatureSensorEvent")

### The DataFrame style

In [47]:
q0bis = (temperature_sdf
                     .where("temperature > 50") # you can add anything that fits in a SQL where statemente 
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

let's ask for the explanation of the plan. Comparing with the one of the SQL style, you can see that there is no difference. This is expected because the [catalyst optimizer](https://databricks.com/glossary/catalyst-optimizer) created it out of our declarations (which are semantically equivalent)

In [48]:
q0bis.explain()

== Physical Plan ==
WriteToDataSourceV2 org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@36f62c60
+- Project [from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#333 as string), Some(Etc/UTC)).sensor AS sensor#348, from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#333 as string), Some(Etc/UTC)).temperature AS temperature#349, from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#333 as string), Some(Etc/UTC)).ts AS ts#350]
   +- Filter (from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#333 as string), Some(Etc/UTC)).temperature > 50.0)
      +- *(1) Project [key#332, value#333, topic#334, partition#335, offset#336L, timestamp#337, timesta

In [49]:
spark.sql("SELECT * FROM sinkTable ORDER BY TS DESC").show(5)

+------+------------------+-------------------+
|sensor|       temperature|                 ts|
+------+------------------+-------------------+
|    S1| 56.29651146995424|2024-10-14 09:00:09|
|    S1| 54.61087713267025|2024-10-14 08:59:59|
|    S1|54.479731860672885|2024-10-14 08:59:49|
|    S1| 55.70282805507498|2024-10-14 08:59:39|
|    S1| 57.08978801827377|2024-10-14 08:59:29|
+------+------------------+-------------------+
only showing top 5 rows



In [50]:
q0bis.stop()

> NOTE: there was no need to
> * create a logic table on top of the streaming data frame with `temperature_sdf.createTempView("TemperatureSensorEvent")`
> * drop such a logic table with `spark.catalog.dropTempView("TemperatureSensorEvent")`

## Q1 - Avg

the average of all the temperature observation for each sensor up to the last event received

### the SQL sytyle

In [51]:
# create a logic table on top of the streaming data frame
temperature_sdf.createTempView("TemperatureSensorEvent") # this time we will not clean it up, because we use it in the next queries

**NOTE**: the following query gives *intentionally* an error

In [52]:
query_string = """
SELECT SENSOR, AVG(temperature) 
FROM TemperatureSensorEvent
GROUP BY SENSOR
"""

# write your query in SQL, register it and start it
q1 = (spark.sql(query_string)
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

AnalysisException: Append output mode not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark;;
Aggregate [SENSOR#348], [SENSOR#348, avg(temperature#349) AS avg(temperature)#734]
+- SubqueryAlias temperaturesensorevent
   +- Project [value#346.sensor AS sensor#348, value#346.temperature AS temperature#349, value#346.ts AS ts#350]
      +- Project [from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#333 as string), Some(Etc/UTC)) AS value#346]
         +- StreamingRelationV2 org.apache.spark.sql.kafka010.KafkaSourceProvider@e9dff0f, kafka, org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaTable@1dbb506c, org.apache.spark.sql.util.CaseInsensitiveStringMap@24f684c4, [key#332, value#333, topic#334, partition#335, offset#336L, timestamp#337, timestampType#338], StreamingRelation DataSource(org.apache.spark.sql.SparkSession@5629c8,kafka,List(),None,List(),No

The **append output mode** (i.e., the default one) is not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark, we need to use the **complete output mode**.

In [68]:
query_string = """
SELECT SENSOR, AVG(temperature) 
FROM TemperatureSensorEvent
GROUP BY SENSOR
"""

# write your query in SQL, register it and start it
q1 = (spark.sql(query_string)
                     .writeStream
                     .format("memory")
                     .outputMode("complete") # <-- CHANGE HERE
                     .queryName("sinkTable")
                     .start())

In [71]:
q1.status

{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

In [72]:
# look up the most recent results
spark.sql("SELECT * FROM sinkTable").show() # without ORDER BY TS DESC because the result in the table is already only the most recent

+------+------------------+
|SENSOR|  avg(temperature)|
+------+------------------+
|    S1|28.869944640999428|
+------+------------------+



**NOTE**: if the cell above gives an empty result, wait 10 seconds and run it again. The very first excution may take time, expecially if you have already ingested many temperature events in kafka. Here we are querying the sink table and it may be empty because the first execution is still running.

In [73]:
# clean up
q1.stop()

### The DataFrame style

In [74]:
# write your query in SQL, register it and start it
q1bis = (temperature_sdf 
                     .groupBy("sensor")
                     .avg()
                     .writeStream
                     .format("memory")
                     .outputMode("complete") 
                     .queryName("sinkTable")
                     .start())

In [75]:
q1bis.status

{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

In [76]:
# look up the most recent results
spark.sql("SELECT * FROM sinkTable").show() # woithout ORDER BY TS DESC because the result in the table is already only the most recent

+------+------------------+
|sensor|  avg(temperature)|
+------+------------------+
|    S1|29.077655376640042|
+------+------------------+



In [77]:
# clean up
q1bis.stop()

## Q2 - Logical Sliding Window

The average temperature observed by each sensor in the last 4 seconds

MEMO: the average should change as soon as the receive a new event

**Not supported**

## Q3 - Logical Tumbling Window

The average temperature of the last 30 seconds every 30 seconds (was 4 seconds in EPL)

NOTE: this query is not possibile in the SQL style

In [78]:
q3 = (temperature_sdf
                  .groupBy(window("TS", "30 seconds"),"SENSOR")
                  .avg("TEMPERATURE")
                  .writeStream
                  .outputMode("complete")
                  .format("memory")
                  .queryName("sinkTable")
                  .start())

In [82]:
q3.status

{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

In [83]:
spark.sql("SELECT * FROM sinkTable ORDER BY window DESC").show(5,False) # NOTE: here we order by window instead of ordering by timestamp# window instead of timestamp, again

+------------------------------------------+------+-----------------+
|window                                    |SENSOR|avg(TEMPERATURE) |
+------------------------------------------+------+-----------------+
|[2024-10-14 09:18:30, 2024-10-14 09:19:00]|S1    |55.28050703602908|
|[2024-10-14 09:18:00, 2024-10-14 09:18:30]|S1    |55.05007715222903|
|[2024-10-14 09:17:30, 2024-10-14 09:18:00]|S1    |54.99187451568674|
|[2024-10-14 09:17:00, 2024-10-14 09:17:30]|S1    |54.28280857790395|
|[2024-10-14 09:16:30, 2024-10-14 09:17:00]|S1    |54.13724108251086|
+------------------------------------------+------+-----------------+
only showing top 5 rows



In [85]:
q3.stop()

## Q4 - Physical Sliding Window

The moving average of the last 4 temperature events

**Not supported**

## Q5 - Physical Tumbling Window

The moving average of the last 4 temperature events every 4 events 

**Not supported**

## Q6 - Logical Hopping Window

The average temperature of the last 1 minute (was 4 seconds in EPL) every 30 seconds (was 2 seconds in EPL)

In [86]:
q6 = (temperature_sdf
      .groupBy(window("TS", "1 minutes", "30 seconds"),"SENSOR")
      .avg("TEMPERATURE")
      .writeStream
      .outputMode("complete")
      .format("memory")
      .queryName("sinkTable")
      .start())

In [89]:
q6.status

{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

In [90]:
spark.sql("SELECT * FROM sinkTable ORDER BY window DESC").show(6,False) # NOTE: here we order by window instead of ordering by timestamp

+------------------------------------------+------+------------------+
|window                                    |SENSOR|avg(TEMPERATURE)  |
+------------------------------------------+------+------------------+
|[2024-10-14 09:20:00, 2024-10-14 09:21:00]|S1    |54.8233050442516  |
|[2024-10-14 09:19:30, 2024-10-14 09:20:30]|S1    |55.14603216241507 |
|[2024-10-14 09:19:00, 2024-10-14 09:20:00]|S1    |54.78651943605533 |
|[2024-10-14 09:18:30, 2024-10-14 09:19:30]|S1    |54.88930406773331 |
|[2024-10-14 09:18:00, 2024-10-14 09:19:00]|S1    |55.30841499505451 |
|[2024-10-14 09:17:30, 2024-10-14 09:18:30]|S1    |55.020975833957884|
+------------------------------------------+------+------------------+
only showing top 6 rows



In [91]:
q6.stop()

## Q7 - Stream-to-Stream Join

In EPL, at this point we moved on to the pattern matching part required to satisfy the information need, i.e., "find every smoke event followed by a temperature event whose temperature is above 50 °C within 2 minutes."

Spark Structured Streaming does not support the EPL's operator `->` (that reads as *followed by*. We need to use a stream-to-stream join.

In [92]:
last_minute_smoke_events = (smoke_sdf
                .where("smoke = True")
                .withColumnRenamed("sensor","sensorSmoke")
                .withColumnRenamed("ts","tsSmoke")
               )

last_minute_high_temperature_events = (temperature_sdf
                .where("temperature > 50")
                .withColumnRenamed("sensor","sensorTemp")
                .withColumnRenamed("ts","tsTemp")
               )

Join with event-time constraints

In [93]:
join_sdf = (last_minute_smoke_events.join(
  last_minute_high_temperature_events, expr("""
    (sensorTemp == sensorSmoke) AND
    (tsTemp > tsSmoke ) AND
    (tsTemp < tsSmoke + interval 2 minute )
    """
    )))

In [94]:
q7 = (join_sdf
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

In [98]:
q7.status

{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

**IMPORTANT** To detect fire, run the appropriate cells in the data generators.

In [100]:
spark.sql("SELECT * FROM sinkTable ORDER BY tsTemp DESC").show(20,False) # note, I change ts in tsTemp

+-----------+-----+-------------------+----------+------------------+-------------------+
|sensorSmoke|smoke|tsSmoke            |sensorTemp|temperature       |tsTemp             |
+-----------+-----+-------------------+----------+------------------+-------------------+
|S1         |true |2024-10-14 09:20:54|S1        |53.94999147704627 |2024-10-14 09:21:24|
|S1         |true |2024-10-14 09:21:04|S1        |53.94999147704627 |2024-10-14 09:21:24|
|S1         |true |2024-10-14 09:21:14|S1        |53.94999147704627 |2024-10-14 09:21:24|
|S1         |true |2024-10-14 09:20:54|S1        |55.52391336827032 |2024-10-14 09:21:14|
|S1         |true |2024-10-14 09:21:04|S1        |55.52391336827032 |2024-10-14 09:21:14|
|S1         |true |2024-10-14 09:20:54|S1        |55.182048128613545|2024-10-14 09:21:04|
+-----------+-----+-------------------+----------+------------------+-------------------+



If you are following carefully the instructions the answer should be empty because we are sending temperature around 55°C, but there is no `smoke==true`, yet.

Go to the `smoke_sensor_simulator` notebook and start sending `smoke==true`.

In [101]:
spark.sql("SELECT * FROM sinkTable ORDER BY tsTemp DESC").show(20,False) # note, I change ts in tsTemp

+-----------+-----+-------------------+----------+------------------+-------------------+
|sensorSmoke|smoke|tsSmoke            |sensorTemp|temperature       |tsTemp             |
+-----------+-----+-------------------+----------+------------------+-------------------+
|S1         |true |2024-10-14 09:20:54|S1        |54.24985043108283 |2024-10-14 09:21:34|
|S1         |true |2024-10-14 09:21:04|S1        |54.24985043108283 |2024-10-14 09:21:34|
|S1         |true |2024-10-14 09:21:14|S1        |54.24985043108283 |2024-10-14 09:21:34|
|S1         |true |2024-10-14 09:21:24|S1        |54.24985043108283 |2024-10-14 09:21:34|
|S1         |true |2024-10-14 09:20:54|S1        |53.94999147704627 |2024-10-14 09:21:24|
|S1         |true |2024-10-14 09:21:04|S1        |53.94999147704627 |2024-10-14 09:21:24|
|S1         |true |2024-10-14 09:21:14|S1        |53.94999147704627 |2024-10-14 09:21:24|
|S1         |true |2024-10-14 09:21:04|S1        |55.52391336827032 |2024-10-14 09:21:14|
|S1       

let's have a look to the progresses

In [ ]:
from IPython.display import clear_output
import json
while True:
    print(json.dumps(q7.lastProgress, indent=4))
    print(q7.status)
    time.sleep(1)
    clear_output(wait=True)
    

to interrupt the execution of the cell, prese the square icon in the bar or choose *interrupt kernel* from the *kernel* dropdown menu

#### Discussion

> This query is equivalent to the EPL pattern `every a = SmokeSensorEvent(smoke=true) -> every TemperatureSensorEvent(temperature > 50, sensor=a.sensor) where timer:within(1 min)`. 
>
> Do not expect the same performances! It is evaluated as a relational join. Spark Structured Streaming lacks the specilized data structure of Esper.
>
> **It does not tame the torrent effect**, but this is expected! 
>
> Spark Structured Streaming is a Data Stream Management System meant to tame *flow that you cannot stop*

Even id Q8 consumes Q7 results, we can stop Q7 because we only need the streaming Data Frame `join_sdf`. We do not need Q7 to write its result in the in memory table.

In [102]:
q7.stop()

## Q8 - Count FireEvent

we are very close to the solution of the running example, we "just" need to count the number of events generated by the previous query over an hopping window of 1 minutes that slides every 30 seconds (was a sliding window of 10 secondsin EPL). 

So let's count the results of Q7. 

**NOTE**: the following queries give *intentionally* errors

In [103]:
q8 = (join_sdf
            .groupBy(window("tsTemp", "1 minutes", "30 seconds"),"sensorTemp")
            .count()
            .writeStream
            .outputMode("complete")
            .format("memory")
            .queryName("sinkTable") 
            .start())

AnalysisException: Join between two streaming DataFrames/Datasets is not supported in Complete output mode, only in Append output mode;;
Join Inner, (((sensorTemp#62740 = sensorSmoke#62732) AND (tsTemp#62744 > tsSmoke#62736)) AND (tsTemp#62744 < cast(tsSmoke#62736 + 2 minutes as timestamp)))
:- Project [sensorSmoke#62732, smoke#320, ts#321 AS tsSmoke#62736]
:  +- Project [sensor#319 AS sensorSmoke#62732, smoke#320, ts#321]
:     +- Filter (smoke#320 = true)
:        +- Project [value#317.sensor AS sensor#319, value#317.smoke AS smoke#320, value#317.ts AS ts#321]
:           +- Project [from_json(StructField(sensor,StringType,true), StructField(smoke,BooleanType,true), StructField(ts,TimestampType,true), cast(value#235 as string), Some(Etc/UTC)) AS value#317]
:              +- StreamingRelationV2 org.apache.spark.sql.kafka010.KafkaSourceProvider@7274de46, kafka, org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaTable@74819d78, org.apache.spark.sql.util.CaseInsensitiveStringMap@7098

Indeed, join between two streaming DataFrames/Datasets is not supported in Complete output mode, only in Append output mode. So let's try to use append mode.

In [104]:
q8 = (join_sdf
            .groupBy(window("tsTemp", "1 minutes", "30 seconds"),"sensorTemp")
            .count()
            .writeStream
            .outputMode("append") # <-- CHANGE HERE
            .format("memory")
            .queryName("sinkTable") 
            .start())

AnalysisException: Append output mode not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark;;
Aggregate [window#113053, sensorTemp#62740], [window#113053 AS window#113044, sensorTemp#62740, count(1) AS count#113052L]
+- Filter ((tsTemp#62744 >= window#113053.start) AND (tsTemp#62744 < window#113053.end))
   +- Expand [ArrayBuffer(named_struct(start, precisetimestampconversion(((((CASE WHEN (cast(CEIL((cast((precisetimestampconversion(tsTemp#62744, TimestampType, LongType) - 0) as double) / cast(30000000 as double))) as double) = (cast((precisetimestampconversion(tsTemp#62744, TimestampType, LongType) - 0) as double) / cast(30000000 as double))) THEN (CEIL((cast((precisetimestampconversion(tsTemp#62744, TimestampType, LongType) - 0) as double) / cast(30000000 as double))) + cast(1 as bigint)) ELSE CEIL((cast((precisetimestampconversion(tsTemp#62744, TimestampType, LongType) - 0) as double) / cast(30000000 as double))) END + cast(0 as bigi

Append output mode is not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark. Indeed, **the streaming join can create out of orders**.

Let's add a watermark, then.

In [105]:
q8 = (join_sdf
            .withWatermark(delayThreshold="2 minutes",eventTime="tsTemp") # <-- CHANGE HERE
            .groupBy(window("tsTemp", "1 minutes", "30 seconds"),"sensorTemp")
            .count()
            .writeStream
            .outputMode("append") 
            .format("memory")
            .queryName("sinkTable") 
            .start())

NOTE: 2 minutes is maximum delay that the join can cause given the way we declared it. The temperature and the smoke event cannot be more than 2 minutes apart.

**THUS, BE PATIENT! YOU'RE NOT GOING TO GET ANY RESULT BEFORE 2 MINUTES HAS PASSED**

In [107]:
spark.sql("SELECT * FROM sinkTable ORDER BY window DESC").show(5,False)

+------------------------------------------+----------+-----+
|window                                    |sensorTemp|count|
+------------------------------------------+----------+-----+
|[2024-10-14 09:22:30, 2024-10-14 09:23:30]|S1        |65   |
|[2024-10-14 09:22:00, 2024-10-14 09:23:00]|S1        |56   |
|[2024-10-14 09:21:30, 2024-10-14 09:22:30]|S1        |39   |
|[2024-10-14 09:21:00, 2024-10-14 09:22:00]|S1        |21   |
|[2024-10-14 09:20:30, 2024-10-14 09:21:30]|S1        |6    |
+------------------------------------------+----------+-----+



In [108]:
q8.stop()